# Evaluation of MFCC

In [2]:
import numpy as np
import scipy.io.wavfile
import scipy.signal
import matplotlib.pyplot as plt
import seaborn as sns
from python_speech_features import mfcc
%matplotlib inline

In [3]:
filename = 'ba_example'
sr, tutor = scipy.io.wavfile.read('../data/{}.wav'.format(filename))
sr, ba_learnt = scipy.io.wavfile.read('../csynthesizer/song_r3.III.wav')

In [13]:
t_mf = mfcc(tutor, samplerate=sr, numcep=4)

In [14]:
print(np.min(t_mf, axis=0))
print(np.max(t_mf, axis=0))
print(np.max(t_mf, axis=0) - np.min(t_mf, axis=0))

[ 10.21074206 -29.16139319 -96.01472663 -13.67298071]
[ 20.47607117   5.54361949 -31.15721375  52.20406423]
[ 10.26532911  34.70501268  64.85751288  65.87704494]


In [5]:
help(mfcc)

Help on function mfcc in module python_speech_features.base:

mfcc(signal, samplerate=16000, winlen=0.025, winstep=0.01, numcep=13, nfilt=26, nfft=512, lowfreq=0, highfreq=None, preemph=0.97, ceplifter=22, appendEnergy=True, winfunc=<function <lambda> at 0x7f9fbf11ec80>)
    Compute MFCC features from an audio signal.
    
    :param signal: the audio signal from which to compute features. Should be an N*1 array
    :param samplerate: the samplerate of the signal we are working with.
    :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)
    :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)
    :param numcep: the number of cepstrum to return, default 13
    :param nfilt: the number of filters in the filterbank, default 26.
    :param nfft: the FFT size. Default is 512.
    :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
    :param highfreq: highest band edge of mel filte